In [26]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
 
#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False
 
#引入TA-Lib库
import talib as ta
import time
from datetime import datetime, timedelta
# import tushare as ts
# df=ts.get_k_data('sh',start='2000-01-01')
# df.index=pd.to_datetime(df.date)
# df=df.sort_index()
# df['ret']=df.close/df.close.shift(1)-1
# df.head()
import akshare as ak

df = ak.stock_zh_a_hist(symbol="000001", adjust="hfq").iloc[:, :6]
    # 处理字段命名，以符合 Backtrader 的要求
df.columns = [
    'date',
    'open',
    'close',
    'high',
    'low',
    'volume',
    ]
df.index=pd.to_datetime(df.date)
df=df.sort_index()

In [27]:
df['ret']=df.close/df.close.shift(1)-1
df['fast_sma'] = ta.SMA(df['close'],timeperiod=5)
df['slow_sma'] = ta.SMA(df['close'],timeperiod=20)
df.dropna()
df = df.loc['2024-01-01':'2024-05-20'] 

In [28]:
df['pos']=0
df['pos'][df['fast_sma']>df['slow_sma']] = 1
df['pos'][df['fast_sma']<=df['slow_sma']] = 0 
df['pos'] = df['pos'].shift(1).fillna(0)
df['pos'].fillna(method='ffill',inplace=True)

/var/folders/vx/c4klfm7j3xb7tpn1qpyjg7300000gn/T/com.apple.shortcuts.mac-helper/ipykernel_76933/2467329736.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['pos'][df['fast_sma']>df['slow_sma']] = 1
/var/folders/vx/c4klfm7j3xb7tpn1qpyjg

In [29]:
df.head()

,date,open,close,high,low,volume,ret,fast_sma,slow_sma,pos
date,,,,,,,,,,
2020-01-02,2020-01-02,2810.25,2846.00,2859.00,2793.99,1530232,0.024574,2801.792,2731.4995,0.0
2020-01-03,2020-01-03,2857.38,2896.38,2917.51,2854.13,1116195,0.017702,2824.870,2745.7205,1.0
2020-01-06,2020-01-06,2868.75,2878.51,2922.39,2852.50,862084,-0.006170,2839.174,2757.6665,1.0
2020-01-07,2020-01-07,2888.26,2891.51,2912.64,2859.00,728608,0.004516,2858.028,2771.8065,1.0
2020-01-08,2020-01-08,2867.13,2811.87,2875.26,2806.99,847824,-0.027543,2864.854,2782.6145,1.0


In [31]:
df['equity']=(df['ret']*df['pos']+1).cumprod() 

In [33]:
# 如果每次只买一手
df['diff']=df.close-df.close.shift(1)
df['equity_1'] = (df['diff']*df['pos']).cumsum() 
df.enew

date
2020-01-02        NaN
2020-01-03      50.38
2020-01-06      32.51
2020-01-07      45.51
2020-01-08     -34.13
               ...   
2024-05-14   -1397.76
2024-05-15   -1410.76
2024-05-16   -1355.50
2024-05-17   -1314.87
2024-05-20   -1321.37
Name: equity_1, Length: 1059, dtype: float64

In [44]:
df.loc['2024-03':'2024-04-22']

,date,open,close,high,low,volume,ret,fast_sma,slow_sma,pos,equity,diff,equity_1
date,,,,,,,,,,,,,
2024-03-01,2024-03-01,1973.39,1957.14,1975.02,1947.39,1828103,-0.008235,1962.014,1878.1490,1.0,0.645199,-16.25,-1327.87
2024-03-04,2024-03-04,1950.64,1931.13,1958.76,1929.51,1655930,-0.013290,1955.512,1883.9185,1.0,0.636624,-26.01,-1353.88
2024-03-05,2024-03-05,1926.26,1947.39,1953.89,1919.76,1817319,0.008420,1953.238,1889.8510,1.0,0.641984,16.26,-1337.62
2024-03-06,2024-03-06,1942.51,1931.13,1950.64,1931.13,1345640,-0.008350,1948.036,1896.5955,1.0,0.636624,-16.26,-1353.88
2024-03-07,2024-03-07,1931.13,1939.26,1981.52,1931.13,2016166,0.004210,1941.210,1904.0720,1.0,0.639304,8.13,-1345.75
2024-03-08,2024-03-08,1934.38,1939.26,1949.01,1926.26,1113974,0.000000,1937.634,1911.9545,1.0,0.639304,0.00,-1345.75
2024-03-11,2024-03-11,1939.26,1953.89,1953.89,1932.76,1210673,0.007544,1942.186,1921.7065,1.0,0.644127,14.63,-1331.12
2024-03-12,2024-03-12,1955.51,1968.52,1973.39,1944.14,1641262,0.007488,1946.412,1931.2960,1.0,0.648950,14.63,-1316.49
2024-03-13,2024-03-13,1963.64,1931.13,1966.89,1926.26,1768039,-0.018994,1946.412,1936.5780,1.0,0.636624,-37.39,-1353.88


In [51]:
-1950.64+1942.51

-8.13000000000011

In [34]:
-2716.6/2

-1358.3

In [53]:
-1953.89+1942.51-1975.02+1923.01-1989.64+2108.29

55.25999999999976

In [54]:
-1968.52+1916.51

-52.00999999999999